## ETL

En este caso trabajas en una empresa de venta al por menor de productos italianos y debes realizar la limpieza, transformación e integración de datos de ventas, productos y clientes para su análisis.

1. Lectura de la Información:

    - Leer los archivos CSV (ventas.csv, productos.csv, clientes.csv).

    - Explorar los conjuntos de datos para comprender su estructura, columnas, tipos de datos, etc.

2. Transformación de Datos:

    - Limpiar los datos: manejar valores nulos, eliminar duplicados si los hay, corregir errores tipográficos, etc.

    - Realizar la integración de datos: unir los conjuntos de datos apropiados para obtener una tabla única que contenga información de ventas junto con detalles de productos y clientes.

    - Aplicar transformaciones relevantes según sea necesario: por ejemplo, convertir tipos de datos, renombrar columnas, crear nuevas características derivadas, etc.